# Numpy Array Operations:  Axes and Broadcasting 

There is an excellent introduction to `numpy` multi-dimensional arrays on the [scipy](https://docs.scipy.org/doc/numpy-dev/user/quickstart.html) website.  In this note, we cover two concepts in a little more detail:
* Using the `axis` feature 
* Python broadcasting

We will need both of these for performing many of the numerical operations for the ML class.

As usual, we begin by loading the `numpy` package.

In [2]:
import numpy as np

## Axis Parameter

Many operations in the `numpy` package can take an optional `axis` parameter to specify which dimensions the operation is to be applied.  This is extremely useful for multi-dimensional data.  To illustrate the `axis` parameter, consider a matrix the `(3,2)` array `X` defined as:

In [3]:
X = np.arange(6).reshape(3,2)
print(X)

[[0 1]
 [2 3]
 [4 5]]


An operation like `np.mean` or `np.sum` takes the mean or sum of *all* elements in the array. 

In [4]:
print(np.mean(X))
print(np.sum(X))

2.5
15


To take only the `sum` along each column, we can use the `axis` parameter.

In [5]:
print(np.sum(X,axis=0))

[6 9]


Since `X` has shape `(3,2)`, the output `np.sum(X,axis=0)` is of shape `(2,)`.  Similarly, we can take the `sum` along each row:

In [6]:
print(np.sum(X,axis=1))

[1 5 9]


You can apply this to higher-order arrays:

In [7]:
X = np.arange(24).reshape(2,3,4)  # shape = (2,3,4)
print(X)
print('\n')
Y1 = np.sum(X,axis=0)             # shape = (3,4)
Y2 = np.sum(X,axis=1)             # shape = (2,4)
print('Y1 = ')
print(Y1)
print('Y2 = ')
print(Y2)

[[[ 0  1  2  3]
  [ 4  5  6  7]
  [ 8  9 10 11]]

 [[12 13 14 15]
  [16 17 18 19]
  [20 21 22 23]]]


Y1 = 
[[12 14 16 18]
 [20 22 24 26]
 [28 30 32 34]]
Y2 = 
[[12 15 18 21]
 [48 51 54 57]]


## Broadcasting

**Broadcasting** is a powerful tool in Python for performing operations on matrices that we will use throughout the ML class.  A good tutorial on broadcasting can be found on the [scipy broadcasting page](https://docs.scipy.org/doc/numpy-1.13.0/user/basics.broadcasting.html).  Here we provide some examples.   

### Example 1:  Mean Removal

Suppose that `X` is a data matrix of shape `(n,p)`.  That is, there are `n` data points and `p` features per point.  Often, we have to remove the mean from each feature.  That is, we want to compute the mean for each feature and then remove the mean from each column.  We could do this with a for-loop as:
   
    Xm = np.zeros(p)      # Mean for each feature
    X1_demean = np.zeros((n,p))  # Transformed features with the means removed
    for j in range(p):
       Xm[j] = np.mean(X[:,j])
       for i in range(n):
           X_demean[i,j] = X[i,j] - Xm[j]
           
The code below does this without a for loop using the `axis` parameter and broadcasting.

In [8]:
# Generate some random data
n = 100
p = 5
X = np.random.rand(n,p)
print('\n')

# Compute the mean per column using the axis command
Xm = np.mean(X,axis=0)  # This is a p-dim matrix

# Subtract the mean
X_demean = X - Xm[None,:]


The command `Xm = np.mean(X,axis=0)` computes the mean of each column which is a `p` dimensional array.  Then, `Xm[None,:]` converts this to a `(1,p)` shape array.  Using Python broadcasting, we can then subtract the `Xm[None,:]` from `X`.

How does this work? First, we use `None` keyword to reshape the matrices `X` and `Y` to compatible sizes

     X[:,None,:]    # Shape nx,  1, p
     Y[None,:,:]    # Shape 1,  ny, p
     
The two matrices can be subtracted so that

     DXY[i,j,k]  = X[i,k] - Y[j,k]
     
Then, `d[i,j] = sum_k (X[i,k] - Y[j,k])**2`, which is the norm squared of the vector differences.

### Example 4:  Outer product

The *outer product* of vectors `x` and `y` is the matrix `Z[i,j] = x[i]y[j]`.  This can be performed in one line as follows

In [9]:
# Some random data
nx = 100
ny = 10
x = np.random.rand(nx)

y = np.random.rand(ny)
print(y)
print('\n')
print(y[None,:])
print('\n')

# Compute the outer product in one line
Z = x[:,None]*y[None,:]
print(Z)


[0.02761219 0.96268886 0.65946218 0.409405   0.30414179 0.14642236
 0.10701401 0.613041   0.24289081 0.92670849]


[[0.02761219 0.96268886 0.65946218 0.409405   0.30414179 0.14642236
  0.10701401 0.613041   0.24289081 0.92670849]]


[[0.0025399  0.08855281 0.06066054 0.03765906 0.02797644 0.01346864
  0.00984367 0.0563905  0.02234228 0.08524316]
 [0.01378325 0.48054815 0.32918562 0.20436386 0.15181933 0.07309007
  0.05341849 0.30601343 0.1212445  0.46258772]
 [0.01391319 0.4850783  0.33228887 0.20629041 0.15325054 0.07377909
  0.05392207 0.30889823 0.12238748 0.46694856]
 [0.02519187 0.87830533 0.60165768 0.37351901 0.27748255 0.13358786
  0.0976338  0.55930551 0.22160046 0.84547878]
 [0.00877134 0.30580944 0.20948592 0.13005231 0.09661422 0.04651279
  0.03399426 0.19473968 0.07715712 0.29437985]
 [0.00814341 0.28391708 0.19448919 0.1207421  0.08969778 0.04318302
  0.03156067 0.18079861 0.07163358 0.27330571]
 [0.00608499 0.21215104 0.14532794 0.09022198 0.06702477 0.0322676
  0.023583

Here:

     x[:,None] # Has shape (nx,  1)
     y[None,:] # Has shape ( 1, ny)
     
So, with python broadcasting:

     Z = x[:,None]*y[None,:] # has shape (nx,  ny)


**Exercise 1:**  Given a matrix `X`, compute the matrix `Y`, where the rows of `X` are normalized to one.  That is:

     Y[i,j] = X[i,j] / sum_j X[i,j]   

In [14]:
X = np.random.rand(4,3)
print(X)
row_sum = X.sum(axis=1)

Y = X / row_sum[:,None]
print(Y)
print('\n')
print(Y.sum(axis=1))

[[0.80595576 0.08058333 0.48428548]
 [0.60948638 0.46068288 0.63572368]
 [0.16304498 0.20237097 0.13563663]
 [0.9590542  0.13732674 0.04012972]]
[[0.58793501 0.05878457 0.35328042]
 [0.3572829  0.27005381 0.37266329]
 [0.32540493 0.40389168 0.27070339]
 [0.84385852 0.12083189 0.03530958]]


[1. 1. 1. 1.]


**Exercise 2:** Diagonal multiplication.  Given a matrix `X` and a vector `d`, compute `Y = diag(d)*X`.

In [20]:
X = np.random.rand(5,3)
d = np.random.rand(5)
    
print(d)
Y = np.matmul(np.diag(d),X)

[0.75204066 0.43458475 0.5629824  0.54297084 0.12107397]
